In [1]:
import os
import datetime
from typing import Optional
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, DeclarativeBase, Mapped
from sqlalchemy import (
    Column, Integer, String, Text, DateTime, func, 
    cast, literal, text, select, Computed, Index,
)
from sqlalchemy.dialects import postgresql
from sqlalchemy.dialects.postgresql import TSVECTOR, REGCONFIG
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.orm import Session, sessionmaker

In [2]:
def render_query(query):
    """
    Takes a SQLAlchemy query object and returns the fully rendered SQL query
    with all the arguments (bind parameters) inserted.
    
    Args:
        query: SQLAlchemy query object to render.
        
    Returns:
        str: Fully rendered SQL query as a string.
    """
    return str(query.compile(dialect=postgresql.dialect(), compile_kwargs={"literal_binds": True}))

In [3]:
def run_query(engine, query, render=True, scalars=True):
    query_string = render_query(query) if render else str(query)
    print("QUERY::: \n", query_string)
    with Session(engine) as session:
        if scalars:
            res = session.scalars(query)
        else:
            res = session.execute(query)
        print("\nRESULT::: \n", list(res))
        return res, query_string

# Base SQLAlchemy example

## Create a Normal Table

### Create a db Engine

In [4]:
import os
from sqlalchemy import create_engine

engine = create_engine(
    os.environ.get("DATABASE_URL"), 
    echo=False,
)
engine

Engine(postgresql+psycopg://admin:***@db:5432/db)

### Define a Table

In [5]:
from sqlalchemy.orm import DeclarativeBase, mapped_column, Mapped
from sqlalchemy import Column, Integer, String, Text, DateTime, func

class Base(DeclarativeBase):
    pass

class Listing(Base):
    __tablename__ = 'listings'
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(unique=True)
    created_at: Mapped[datetime.datetime] = mapped_column(
        default=func.now())
    updated_at: Mapped[datetime.datetime] = mapped_column(
        default=func.now(), onupdate=func.now())
    
    def __repr__(self):
        return f"<Listing(id='{self.id}' name='{self.name}')>"


Listing.__table__.drop(engine, checkfirst=True)
Base.metadata.create_all(engine)

### Add Some Records

In [6]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    session.add_all([
        Listing(name="Addison"), 
        Listing(name="North")
    ])
    session.commit()

### Show the db Records

In [7]:
%load_ext sql

In [8]:
%%sql $DB_URL
select * from listings;

2 rows affected.


id,name,created_at,updated_at
1,Addison,2024-10-09 01:29:53.960089,2024-10-09 01:29:53.960089
2,North,2024-10-09 01:29:53.960089,2024-10-09 01:29:53.960089


### Query using sqlalchemy

In [9]:
from sqlalchemy import select

query = select(Listing)

_, query_string = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.created_at, listings.updated_at 
FROM listings

RESULT::: 
 [<Listing(id='1' name='Addison')>, <Listing(id='2' name='North')>]


In [10]:
from sqlalchemy import select

query = select(Listing.id, Listing.name)

_, query_string = run_query(engine, query, scalars=False)

QUERY::: 
 SELECT listings.id, listings.name 
FROM listings

RESULT::: 
 [(1, 'Addison'), (2, 'North')]


In [11]:
from sqlalchemy import select

query = select(Listing.id, Listing.name).filter(Listing.name == "Addison")

_, query_string = run_query(engine, query, scalars=False)

QUERY::: 
 SELECT listings.id, listings.name 
FROM listings 
WHERE listings.name = 'Addison'

RESULT::: 
 [(1, 'Addison')]


# JSONB Example

## Create Table with JSONB

### Define a Table with JSONB field

In [12]:
from sqlalchemy.orm import mapped_column, Mapped, DeclarativeBase
from sqlalchemy import Column, Integer, String, Text, DateTime, func
from sqlalchemy.dialects.postgresql import JSONB

class Base(DeclarativeBase):
    pass

class Listing(Base):
    __tablename__ = 'listings'
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(unique=True)
    rooms: Mapped[JSONB] = mapped_column(type_=JSONB)
    created_at: Mapped[datetime.datetime] = mapped_column(
        default=func.now())
    updated_at: Mapped[datetime.datetime] = mapped_column(
        default=func.now(), onupdate=func.now())
    
    def __repr__(self):
        return f"<Listing(id='{self.id}' name='{self.name}')>"


Listing.__table__.drop(engine, checkfirst=True)
Base.metadata.create_all(engine)

### Add Some Records

In [13]:
from sqlalchemy.orm import Session
with Session(engine) as session:
    session.add_all([
        Listing(name="Irving", rooms={
            "kitchen": {"color": "blue"},
            "bathroom": {"color": "green"},
        }), 
        Listing(name="Diversey", rooms={
            "bedroom": {"color": "yellow"},
            "bathroom": {"color": "yellow"},
            "kitchen": {"color": "yellow"},
        }), 
        Listing(name="Main", rooms={
            "bedroom": {"color": "blue"},
            "bathroom": {"color": "green"},
            "basement": {"color": "green"},
        }), 
        Listing(name="vacant", rooms={
            "bathroom": None,
            "bedroom": {"color": "red"},
            "basement": {"color": "green"},
        }), 
        Listing(name="Addison", rooms={
            "basement": {"color": "green"},
            "bedroom": {"color": "green"},
            "bathroom": {"color": "green"},
        }),
        Listing(name="North", rooms={
            "basement": {"color": "green"},
        }),
    ])
    session.commit()

### Show the db Records

In [14]:
%%sql $DB_URL
select id,name,rooms from listings;

6 rows affected.


id,name,rooms
1,Irving,"{'kitchen': {'color': 'blue'}, 'bathroom': {'color': 'green'}}"
2,Diversey,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}, 'bathroom': {'color': 'yellow'}}"
3,Main,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}, 'bathroom': {'color': 'green'}}"
4,vacant,"{'bedroom': {'color': 'red'}, 'basement': {'color': 'green'}, 'bathroom': None}"
5,Addison,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}, 'bathroom': {'color': 'green'}}"
6,North,{'basement': {'color': 'green'}}


### Filter listings with bathroom

In [15]:
from sqlalchemy import select

query = select(Listing).filter(
    Listing.rooms.has_key('bathroom'))

_ = run_query(engine, query, render=True)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE listings.rooms ? 'bathroom'

RESULT::: 
 [<Listing(id='1' name='Irving')>, <Listing(id='2' name='Diversey')>, <Listing(id='3' name='Main')>, <Listing(id='4' name='vacant')>, <Listing(id='5' name='Addison')>]


In [16]:
from sqlalchemy import select

query = select(Listing).filter(
    Listing.rooms.op('?')('bedroom'))

_ = run_query(engine, query, render=True)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE listings.rooms ? 'bedroom'

RESULT::: 
 [<Listing(id='2' name='Diversey')>, <Listing(id='3' name='Main')>, <Listing(id='4' name='vacant')>, <Listing(id='5' name='Addison')>]


In [17]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE listings.rooms ? 'bedroom'

4 rows affected.


id,name,rooms,created_at,updated_at
2,Diversey,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}, 'bathroom': {'color': 'yellow'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
3,Main,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}, 'bathroom': {'color': 'green'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
4,vacant,"{'bedroom': {'color': 'red'}, 'basement': {'color': 'green'}, 'bathroom': None}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
5,Addison,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}, 'bathroom': {'color': 'green'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277


### Filter `bedroom.color == yellow`

In [18]:
from sqlalchemy import select

query = select(Listing).filter(
    Listing.rooms['bedroom']["color"].astext == "yellow",
)
_, query_string = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (((listings.rooms -> 'bedroom')) ->> 'color') = 'yellow'

RESULT::: 
 [<Listing(id='2' name='Diversey')>]


In [19]:
from sqlalchemy import select

query = select(Listing).filter(
    Listing.rooms.op("->")('bedroom').op("->>")('color') == "yellow",
)
_, query_string = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE ((listings.rooms -> 'bedroom') ->> 'color') = 'yellow'

RESULT::: 
 [<Listing(id='2' name='Diversey')>]


In [20]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (((listings.rooms -> 'bedroom')) ->> 'color') = 'yellow'

1 rows affected.


id,name,rooms,created_at,updated_at
2,Diversey,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}, 'bathroom': {'color': 'yellow'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277


### Filter `bedroom.color == yellow`

In [21]:
from sqlalchemy import select

query = select(Listing).filter(
    Listing.rooms['bedroom'] == {"color": "yellow"},
)
_ = run_query(engine, query, render=False)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE listings.rooms[:rooms_1] = :param_1

RESULT::: 
 [<Listing(id='2' name='Diversey')>]


In [22]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at
FROM listings 
WHERE (listings.rooms -> 'bedroom'::TEXT) = '{"color": "yellow"}'::JSONB

1 rows affected.


id,name,rooms,created_at,updated_at
2,Diversey,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}, 'bathroom': {'color': 'yellow'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277


### Filter `bathroom is Not Null`

In [23]:
query = select(Listing).filter(
    func.jsonb_extract_path_text(Listing.rooms, 'bathroom').is_(None)
)
_ = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE jsonb_extract_path_text(listings.rooms, 'bathroom') IS NULL

RESULT::: 
 [<Listing(id='4' name='vacant')>, <Listing(id='6' name='North')>]


In [24]:
query = select(Listing).filter(
    func.jsonb_extract_path_text(Listing.rooms, 'bathroom').isnot(None)
)
_ = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE jsonb_extract_path_text(listings.rooms, 'bathroom') IS NOT NULL

RESULT::: 
 [<Listing(id='1' name='Irving')>, <Listing(id='2' name='Diversey')>, <Listing(id='3' name='Main')>, <Listing(id='5' name='Addison')>]


In [25]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE jsonb_extract_path_text(listings.rooms, 'bathroom') IS NULL

2 rows affected.


id,name,rooms,created_at,updated_at
4,vacant,"{'bedroom': {'color': 'red'}, 'basement': {'color': 'green'}, 'bathroom': None}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
6,North,{'basement': {'color': 'green'}},2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277


### Filter `kitchen and bathroom has a color`

In [26]:
query = select(Listing).filter(
    Listing.rooms.op('->')('kitchen').op('?')('color'))

_ = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (listings.rooms -> 'kitchen') ? 'color'

RESULT::: 
 [<Listing(id='1' name='Irving')>, <Listing(id='2' name='Diversey')>]


In [27]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (listings.rooms -> 'kitchen') ? 'color'

2 rows affected.


id,name,rooms,created_at,updated_at
1,Irving,"{'kitchen': {'color': 'blue'}, 'bathroom': {'color': 'green'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
2,Diversey,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}, 'bathroom': {'color': 'yellow'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277


In [28]:
query = select(Listing).filter(
    Listing.rooms.op('->')('bathroom').op('?')('color'))

_ = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (listings.rooms -> 'bathroom') ? 'color'

RESULT::: 
 [<Listing(id='1' name='Irving')>, <Listing(id='2' name='Diversey')>, <Listing(id='3' name='Main')>, <Listing(id='5' name='Addison')>]


In [29]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (listings.rooms -> 'bathroom') ? 'color'

4 rows affected.


id,name,rooms,created_at,updated_at
1,Irving,"{'kitchen': {'color': 'blue'}, 'bathroom': {'color': 'green'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
2,Diversey,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}, 'bathroom': {'color': 'yellow'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
3,Main,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}, 'bathroom': {'color': 'green'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
5,Addison,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}, 'bathroom': {'color': 'green'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277


### Find all rooms with a green color using JSONPath

In [30]:
query = select(Listing).where(
    func.jsonb_path_exists(
        Listing.rooms, text("""'$.* ? (@.color == "green")'::jsonpath""")))

_ = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE jsonb_path_exists(listings.rooms, '$.* ? (@.color == "green")'::jsonpath)

RESULT::: 
 [<Listing(id='1' name='Irving')>, <Listing(id='3' name='Main')>, <Listing(id='4' name='vacant')>, <Listing(id='5' name='Addison')>, <Listing(id='6' name='North')>]


In [31]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE jsonb_path_exists(listings.rooms, '$.* ? (@.color == "green")'::jsonpath)

5 rows affected.


id,name,rooms,created_at,updated_at
1,Irving,"{'kitchen': {'color': 'blue'}, 'bathroom': {'color': 'green'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
3,Main,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}, 'bathroom': {'color': 'green'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
4,vacant,"{'bedroom': {'color': 'red'}, 'basement': {'color': 'green'}, 'bathroom': None}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
5,Addison,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}, 'bathroom': {'color': 'green'}}",2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277
6,North,{'basement': {'color': 'green'}},2024-10-09 01:30:03.123277,2024-10-09 01:30:03.123277


### Use JSONB operators in the select 

In [32]:
query = select(
    Listing.id, 
    Listing.rooms.op("->>")('bedroom'),
    Listing.rooms.op("->")('bedroom').op("->>")('color'),
)
_ = run_query(engine, query, scalars=False)

QUERY::: 
 SELECT listings.id, listings.rooms ->> 'bedroom' AS anon_1, (listings.rooms -> 'bedroom') ->> 'color' AS anon_2 
FROM listings

RESULT::: 
 [(1, None, None), (2, '{"color": "yellow"}', 'yellow'), (3, '{"color": "blue"}', 'blue'), (4, '{"color": "red"}', 'red'), (5, '{"color": "green"}', 'green'), (6, None, None)]


In [33]:
%%sql $DB_URL 
SELECT listings.id, listings.rooms ->> 'bedroom' AS anon_1, (listings.rooms -> 'bedroom') ->> 'color' AS anon_2 
FROM listings

6 rows affected.


id,anon_1,anon_2
1,None,None
2,"{""color"": ""yellow""}",yellow
3,"{""color"": ""blue""}",blue
4,"{""color"": ""red""}",red
5,"{""color"": ""green""}",green
6,None,None


In [34]:
# https://www.postgresql.org/docs/9.6/functions-json.html
# https://www.postgresql.org/docs/current/datatype-json.html

In [35]:
# https://github.com/json-path/JsonPath

# Search Example

### Generate a "Vectorized" representation of the text

In [36]:
%%sql $DB_URL 
select to_tsvector(
    'my name is Joe and I live in Chicago. I have used Django and kubernetes');

1 rows affected.


to_tsvector
'chicago':9 'django':13 'joe':4 'kubernet':15 'live':7 'name':2 'use':12


### Do the search terms exist in the vector? 

In [37]:
%%sql $DB_URL 
select to_tsvector(
    'my name is Joe and I live in Chicago. I have used Django and kubernetes')
@@ to_tsquery('Chicago & Django');

1 rows affected.


?column?
True


In [38]:
%%sql $DB_URL 
select to_tsvector(
    'my name is Joe and I live in Chicago. I have used Django and kubernetes')
@@ to_tsquery('DC');

1 rows affected.


?column?
False


In [39]:
from sqlalchemy.orm import mapped_column, Mapped, DeclarativeBase
from sqlalchemy.types import TypeDecorator

class Base(DeclarativeBase):
    pass


class TSVector(TypeDecorator):
    impl = TSVECTOR
    cache_ok = True  

class Listing(Base):
    __tablename__ = 'listings'

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(unique=True)
    description: Mapped[Optional[str]] = mapped_column()
    rooms: Mapped[JSONB] = mapped_column(type_=JSONB)
    created_at: Mapped[datetime.datetime] = mapped_column(
        default=func.now())
    updated_at: Mapped[datetime.datetime] = mapped_column(
        default=func.now(), onupdate=func.now())
    
    ts_vector = mapped_column(
        TSVector(), 
        Computed(
            func.setweight(
                func.to_tsvector(
                    cast(literal("english"), type_=REGCONFIG), 
                    func.coalesce(name, '')), 'A' ).concat(
            func.setweight(
                func.to_tsvector(
                    cast(literal("english"), type_=REGCONFIG), 
                    func.coalesce(description, '')), "B" )),
            persisted=True))
    
    __table_args__ = (
        Index('ix_video___ts_vector__', ts_vector, postgresql_using='gin'),)
    
    def __repr__(self):
        return f"<Listing(id='{self.id}' name='{self.name}')>"



Listing.__table__.drop(engine, checkfirst=True)
Base.metadata.create_all(engine)

In [40]:
from sqlalchemy.orm import Session
with Session(engine) as session:
    session.add_all([
        Listing(
            name="home", 
            description="No place like home. A great place to live.",
            rooms={
            "kitchen": {"color": "blue"},
        }), 
        Listing(
            name="away", 
            description="A house that is very yellow and bright themed.",
            rooms={
            "bedroom": {"color": "yellow"},
            "kitchen": {"color": "yellow"},
        }), 
        Listing(
            name="far", 
            description="A large house in the middle of the countryside and has a barnhouse theme.",
            rooms={
            "bedroom": {"color": "blue"},
            "basement": {"color": "green"},
        }), 
        Listing(
            name="city", 
            description="Green 18th century old house with big garage",
            rooms={
            "basement": {"color": "green"},
            "bedroom": {"color": "green"},
        }),
        Listing(
            name="shelter", 
            description="a dark and scary storm shelter",
            rooms={
            "basement": {"color": "green"},
        }),
    ])
    session.commit()

In [41]:
%%sql $DB_URL
select id, name, description, ts_vector from listings;

5 rows affected.


id,name,description,ts_vector
1,home,No place like home. A great place to live.,"'great':7B 'home':1A,5B 'like':4B 'live':10B 'place':3B,8B"
2,away,A house that is very yellow and bright themed.,'away':1A 'bright':9B 'hous':3B 'theme':10B 'yellow':7B
3,far,A large house in the middle of the countryside and has a barnhouse theme.,'barnhous':14B 'countrysid':10B 'far':1A 'hous':4B 'larg':3B 'middl':7B 'theme':15B
4,city,Green 18th century old house with big garage,'18th':3B 'big':8B 'centuri':4B 'citi':1A 'garag':9B 'green':2B 'hous':6B 'old':5B
5,shelter,a dark and scary storm shelter,"'dark':3B 'scari':5B 'shelter':1A,7B 'storm':6B"


In [67]:
from sqlalchemy.orm import Session
search_query = 'yellow house'

query = select(Listing).filter(
    Listing.ts_vector.op('@@')(
        func.plainto_tsquery(
            cast(literal("english"), type_=REGCONFIG), search_query)
    )
)

_ = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.description, listings.rooms, listings.created_at, listings.updated_at, ST_AsEWKB(listings.geo_location) AS geo_location, listings.ts_vector 
FROM listings 
WHERE listings.ts_vector @@ plainto_tsquery(CAST('english' AS REGCONFIG), 'yellow house')

RESULT::: 
 [<Listing(id='2' name='away')>]


In [43]:
%%sql $DB_URL
SELECT listings.id, listings.name, listings.description, listings.rooms, listings.created_at, listings.updated_at, listings.ts_vector 
FROM listings 
WHERE listings.ts_vector @@ plainto_tsquery(CAST('english' AS REGCONFIG), 'house yellow')

1 rows affected.


id,name,description,rooms,created_at,updated_at,ts_vector
2,away,A house that is very yellow and bright themed.,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-09 01:30:35.678744,2024-10-09 01:30:35.678744,'away':1A 'bright':9B 'hous':3B 'theme':10B 'yellow':7B


In [44]:
from sqlalchemy.orm import Session
search_query = 'house | yellow | country'

query = select(Listing).filter(
    Listing.ts_vector.op('@@')(
        func.to_tsquery(cast(literal("english"), type_=REGCONFIG), search_query)
    )
)

_ = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.description, listings.rooms, listings.created_at, listings.updated_at, listings.ts_vector 
FROM listings 
WHERE listings.ts_vector @@ to_tsquery(CAST('english' AS REGCONFIG), 'house | yellow | country')

RESULT::: 
 [<Listing(id='2' name='away')>, <Listing(id='3' name='far')>, <Listing(id='4' name='city')>]


In [45]:
%%sql $DB_URL
SELECT listings.id, listings.name, listings.description, listings.rooms, listings.created_at, listings.updated_at, listings.ts_vector 
FROM listings 
WHERE listings.ts_vector @@ to_tsquery(CAST('english' AS REGCONFIG), 'house | yellow | country');

3 rows affected.


id,name,description,rooms,created_at,updated_at,ts_vector
2,away,A house that is very yellow and bright themed.,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-09 01:30:35.678744,2024-10-09 01:30:35.678744,'away':1A 'bright':9B 'hous':3B 'theme':10B 'yellow':7B
3,far,A large house in the middle of the countryside and has a barnhouse theme.,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}}",2024-10-09 01:30:35.678744,2024-10-09 01:30:35.678744,'barnhous':14B 'countrysid':10B 'far':1A 'hous':4B 'larg':3B 'middl':7B 'theme':15B
4,city,Green 18th century old house with big garage,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}}",2024-10-09 01:30:35.678744,2024-10-09 01:30:35.678744,'18th':3B 'big':8B 'centuri':4B 'citi':1A 'garag':9B 'green':2B 'hous':6B 'old':5B


In [46]:
# https://www.postgresql.org/docs/current/textsearch-intro.html#TEXTSEARCH-DOCUMENT

# GeoSpacial Search Example

In [47]:
with engine.connect() as con:
    con.execute(text("CREATE EXTENSION IF NOT EXISTS postgis"))
    con.commit()

In [48]:
from sqlalchemy.orm import mapped_column, Mapped, DeclarativeBase
from sqlalchemy import Column, Integer, String, Text, DateTime, func
from sqlalchemy.dialects.postgresql import JSONB
from geoalchemy2 import Geometry, WKBElement
from sqlalchemy.types import TypeDecorator

class Base(DeclarativeBase):
    pass


class TSVector(TypeDecorator):
    impl = TSVECTOR
    cache_ok = True  

class Listing(Base):
    __tablename__ = 'listings'

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(unique=True)
    description: Mapped[Optional[str]] = mapped_column()
    rooms: Mapped[JSONB] = mapped_column(type_=JSONB)
    created_at: Mapped[datetime.datetime] = mapped_column(
        default=func.now())
    updated_at: Mapped[datetime.datetime] = mapped_column(
        default=func.now(), onupdate=func.now())
    
    geo_location: Mapped[Optional[WKBElement]] = mapped_column(
        Geometry(geometry_type="POINT", srid=4326, spatial_index=True)
    )
    
    ts_vector = mapped_column(
        TSVector(), 
        Computed(
            func.setweight(
                func.to_tsvector(
                    cast(literal("english"), type_=REGCONFIG), 
                    func.coalesce(name, '')), 'A' ).concat(
            func.setweight(
                func.to_tsvector(
                    cast(literal("english"), type_=REGCONFIG), 
                    func.coalesce(description, '')), "B" )),
            persisted=True))
    
    __table_args__ = (
        Index('ix_video___ts_vector__', ts_vector, postgresql_using='gin'),)
    
    def __repr__(self):
        return f"<Listing(id='{self.id}' name='{self.name}')>"

Listing.__table__.drop(engine, checkfirst=True)
Base.metadata.create_all(engine)

In [49]:
from sqlalchemy.orm import Session
with Session(engine) as session:
    session.add_all([
        Listing(
            name="home", 
            description="No place like home. A great place to live.",
            rooms={
            "kitchen": {"color": "blue"},
            },
            geo_location=f"POINT(-87.671078 41.947965)",
        ), 
        Listing(
            name="away", 
            description="A house that is very yellow and bright themed.",
            rooms={
            "bedroom": {"color": "yellow"},
            "kitchen": {"color": "yellow"},
            },
            geo_location=f"POINT(-87.656563 41.948607)",
        ), 
        Listing(
            name="far", 
            description="A large house in the middle of the countryside and has a barnhouse theme.",
            rooms={
            "bedroom": {"color": "blue"},
            "basement": {"color": "green"},
            },
            geo_location=f"POINT(-87.63834236651081 41.94661707935401)",
        ), 
        Listing(
            name="city", 
            description="Green 18th century old house with big garage",
            rooms={
            "basement": {"color": "green"},
            "bedroom": {"color": "green"},
            },
            geo_location=f"POINT(-87.69332477362649 41.94485677721036)",
        ), 
        Listing(
            name="shelter", 
            description="a dark and scary storm shelter",
            rooms={
            "basement": {"color": "green"},
            },
            geo_location=f"POINT(-87.634333389981 41.87813926214243)",
        ), 
    ])
    session.commit()

In [50]:
%%sql $DB_URL
select * from listings;

5 rows affected.


id,name,description,rooms,created_at,updated_at,geo_location,ts_vector
1,home,No place like home. A great place to live.,{'kitchen': {'color': 'blue'}},2024-10-09 01:30:43.106629,2024-10-09 01:30:43.106629,0101000020E61000002AC423F1F2EA55C05760C8EA56F94440,"'great':7B 'home':1A,5B 'like':4B 'live':10B 'place':3B,8B"
2,away,A house that is very yellow and bright themed.,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-09 01:30:43.106629,2024-10-09 01:30:43.106629,0101000020E6100000E030D12005EA55C0DBE044F46BF94440,'away':1A 'bright':9B 'hous':3B 'theme':10B 'yellow':7B
3,far,A large house in the middle of the countryside and has a barnhouse theme.,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}}",2024-10-09 01:30:43.106629,2024-10-09 01:30:43.106629,0101000020E61000002CF4F099DAE855C08CD49ABF2AF94440,'barnhous':14B 'countrysid':10B 'far':1A 'hous':4B 'larg':3B 'middl':7B 'theme':15B
4,city,Green 18th century old house with big garage,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}}",2024-10-09 01:30:43.106629,2024-10-09 01:30:43.106629,0101000020E6100000DF0EDF6E5FEC55C0E0C21E11F1F84440,'18th':3B 'big':8B 'centuri':4B 'citi':1A 'garag':9B 'green':2B 'hous':6B 'old':5B
5,shelter,a dark and scary storm shelter,{'basement': {'color': 'green'}},2024-10-09 01:30:43.106629,2024-10-09 01:30:43.106629,0101000020E6100000AB2E13EB98E855C01F1E0ADE66F04440,"'dark':3B 'scari':5B 'shelter':1A,7B 'storm':6B"


In [51]:
from geoalchemy2.functions import ST_DWithin, ST_Transform, ST_SetSRID, ST_GeomFromText
from geoalchemy2 import Geography

longitude=-87.634333389981
latitude=41.87813926214243
radius=8000  # in meters

query = select(Listing).where(
    ST_DWithin(
        cast(Listing.geo_location, Geography),
        cast(
            ST_SetSRID(
                func.ST_MakePoint(longitude, latitude),
                4326
            ),
            Geography
        ),
        radius
    )
)

# Execute the query
_ = run_query(engine, query)

QUERY::: 
 SELECT listings.id, listings.name, listings.description, listings.rooms, listings.created_at, listings.updated_at, ST_AsEWKB(listings.geo_location) AS geo_location, listings.ts_vector 
FROM listings 
WHERE ST_DWithin(CAST(listings.geo_location AS geography(GEOMETRY,-1)), CAST(ST_SetSRID(ST_MakePoint(-87.634333389981, 41.87813926214243), 4326) AS geography(GEOMETRY,-1)), 8000)

RESULT::: 
 [<Listing(id='3' name='far')>, <Listing(id='5' name='shelter')>]


In [56]:
from geoalchemy2.functions import ST_DWithin, ST_Transform, ST_SetSRID, ST_GeomFromText, ST_Distance
from geoalchemy2 import Geography

longitude=-87.634333389981
latitude=41.87813926214243
radius=8000  # in meters

query = select(
    Listing.id, 
    Listing.name,
    ST_Distance(
        cast(Listing.geo_location, Geography), 
        cast(
            ST_SetSRID(func.ST_MakePoint(longitude, latitude), 4326),
             Geography
            )
    )
).where(
    ST_DWithin(
        cast(Listing.geo_location, Geography),
        cast(
            ST_SetSRID(
                func.ST_MakePoint(longitude, latitude),
                4326
            ),
            Geography
        ),
        radius
    )
)

# Execute the query
_ = run_query(engine, query, scalars=False)

QUERY::: 
 SELECT listings.id, listings.name, ST_Distance(CAST(listings.geo_location AS geography(GEOMETRY,-1)), CAST(ST_SetSRID(ST_MakePoint(-87.634333389981, 41.87813926214243), 4326) AS geography(GEOMETRY,-1))) AS "ST_Distance_1" 
FROM listings 
WHERE ST_DWithin(CAST(listings.geo_location AS geography(GEOMETRY,-1)), CAST(ST_SetSRID(ST_MakePoint(-87.634333389981, 41.87813926214243), 4326) AS geography(GEOMETRY,-1)), 8000)

RESULT::: 
 [(3, 'far', 7613.20826318), (5, 'shelter', 0.0)]


In [57]:
%%sql $DB_URL
SELECT 
   listings.id, 
   ST_Distance(
       CAST(listings.geo_location AS geography(GEOMETRY,-1)), 
       CAST(ST_SetSRID(ST_MakePoint(-87.634333389981, 41.87813926214243), 4326) AS geography(GEOMETRY,-1))) 
   AS "ST_Distance_1" 
FROM listings 
WHERE ST_DWithin(CAST(listings.geo_location AS geography(GEOMETRY,-1)), CAST(ST_SetSRID(ST_MakePoint(-87.634333389981, 41.87813926214243), 4326) AS geography(GEOMETRY,-1)), 8000)

2 rows affected.


id,ST_Distance_1
3,7613.20826318
5,0.0


# Database Locking

In [54]:
from sqlalchemy import func, select
from sqlalchemy.orm import Session
from contextlib import contextmanager
from dataclasses import dataclass

@dataclass
class Lock:
    locked: bool
    lock_key: int


@contextmanager
def advisory_lock(session: Session, lock_value: str, commit: bool = True):
    """
    Context manager that acquires a PostgreSQL advisory lock using a hash of the lock_value
    upon entering and releases it upon exiting, using SQLAlchemy's syntax.
    
    Args:
        session (Session): SQLAlchemy session to interact with the database.
        lock_value (str): The value (string) to hash and generate the lock key.
    
    Usage:
        with advisory_lock(session, "some_unique_value"):
            # Critical section where the lock is held
    """
    try:
        # Acquire the advisory lock using hashtext and pg_advisory_xact_lock in a single call
        lock_key_query = select(func.hashtext(lock_value))
        lock_key = session.execute(lock_key_query).scalar()
        
        lock = session.execute(
            select(func.pg_try_advisory_xact_lock(lock_key))
        ).first()[0]
        # Yield control back to the caller inside the context
        yield Lock(locked=not lock, lock_key=lock_key)
    finally:
        # The lock is automatically released when the transaction ends (no explicit release needed)
        if commit:
            session.commit()

In [55]:
from sqlalchemy.orm import sessionmaker
import time

LocalSession = sessionmaker(autocommit=False, autoflush=False, bind=engine)

lock_value = "some_unique_string_value"  # This string will be hashed to generate the lock key

with LocalSession() as session:
    with advisory_lock(session, lock_value, commit=True) as lock:
        if lock.locked:
            print("Skip because locked")
        else:
            print("GOT THE LOCK!")
            # Critical section where the advisory lock is held
            # Perform operations that need the lock
            session.execute(text('select 1'))
            time.sleep(30)
            print(lock.locked, lock.lock_key)


GOT THE LOCK!
False 1637609686
